In [1]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [2]:
databases = ['ec2-18-197-150-189.eu-central-1.compute.db',
'ec2-3-91-76-120.compute-1.db',
'ec2-54-153-52-209.us-west-1.compute.db',
'ec2-13-244-77-73.af-south-1.compute.db',
'ec2-13-208-166-38.ap-northeast-3.compute.db',
'ec2-54-79-89-121.ap-southeast-2.compute.db',
'ec2-54-232-145-115.sa-east-1.compute.db']

connections = []
cursors = []
db_pre_string = './misc-measurements/'

for database in databases:
    conn = sq.connect(f"{db_pre_string}{database}")
    conn.row_factory = sq.Row
    connections.append(conn)
    cursors.append(conn.cursor())

In [11]:
def check_commonalities(index_a, db_a, db_b, table, cols):
    attach_sql = f"""
    ATTACH DATABASE '{db_pre_string}{db_b}' AS db_b;
    """
    cursors[index_a].execute(attach_sql)
    connections[index_a].commit()
    
    detach_sql = """
    DETACH DATABASE db_b;
    """
    
    try:
        cols_string = ', '.join(cols)
        main_sql = f"""
        SELECT {cols_string} FROM {table}
        EXCEPT
        SELECT {cols_string} FROM db_b.{table};
        """
        cursors[index_a].execute(main_sql)
        items = list(cursors[index_a].fetchall())
        
    except:
        cursors[index_a].execute(detach_sql)
        connections[index_a].commit()
    cursors[index_a].execute(detach_sql)
    connections[index_a].commit()
    return items

In [12]:
for index_a, db_a in enumerate(databases):
    for db_b in databases:
        if db_a != db_b:
            cols = ['ip', 'port', 'q_version', 'draft_version']
            items = check_commonalities(index_a, db_a, db_b, 'q_versions', cols)
            
            print(f"{db_a.split('.')[1]} with {db_b.split('.')[1]}")
            if items is not None:
                for item in items:
                    for col in cols:
                        print(f"{col}: {item[col]}")

eu-central-1 with compute-1
eu-central-1 with us-west-1
eu-central-1 with af-south-1
ip: 41.223.53.120
port: 8853
q_version: 4278190109
draft_version: doq
ip: 73.79.210.148
port: 784
q_version: 4278190114
draft_version: doq-i02
ip: 82.65.248.221
port: 784
q_version: 4278190114
draft_version: doq-i02
eu-central-1 with ap-northeast-3
ip: 114.204.32.239
port: 784
q_version: 4278190114
draft_version: doq-i02
ip: 82.65.248.221
port: 784
q_version: 4278190114
draft_version: doq-i02
eu-central-1 with ap-southeast-2
ip: 82.65.248.221
port: 784
q_version: 4278190114
draft_version: doq-i02
eu-central-1 with sa-east-1
ip: 114.204.32.239
port: 784
q_version: 4278190114
draft_version: doq-i02
ip: 82.65.248.221
port: 784
q_version: 4278190114
draft_version: doq-i02
compute-1 with eu-central-1
ip: 103.254.108.74
port: 784
q_version: 4278190114
draft_version: doq-i02
ip: 35.198.139.194
port: 784
q_version: 4278190114
draft_version: doq-i02
compute-1 with us-west-1
ip: 103.254.108.74
port: 784
q_versio

In [13]:
for cur in cursors:
    cur.close()
for conn in connections:
    conn.close()